In [58]:
import ipywidgets as widgets
from fastai.vision.all import *

In [59]:
path = '../input/stanford-dogs-dataset/images'

In [60]:
dogs = DataBlock(blocks=(ImageBlock, CategoryBlock),
                get_items=get_image_files,
                splitter=RandomSplitter(valid_pct=0.2, seed=42),
                get_y=parent_label,
                item_tfms=Resize(128),
                batch_tfms=aug_transforms(size=224))

In [61]:
dls = dogs.dataloaders(path)

In [62]:
dls.show_batch()

In [63]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

In [68]:
btn_upload = widgets.FileUpload()
def button_click_func(change):
    btn_upload.value.clear()
    btn_upload._counter = 1
btn_upload.observe(button_click_func)
btn_upload

In [65]:
img = PILImage.create(btn_upload.data[-1])
img

In [69]:
learn.export()

In [70]:
learn_inf = load_learner(Path()/'export.pkl')

In [71]:
lbl_pred = widgets.Label()
lbl_pred

In [72]:
out_pl = widgets.Output()
out_pl

In [73]:
btn_run = widgets.Button(description="Classify")

def edit_lbl(lbl):
    new_lbl = ""
    for c in lbl.lstrip('n'+''.join(list(map(str, range(10))))+'-'):
        if c == '-' or c == '_':
            new_lbl += ' '
        else:
            new_lbl += c
    return new_lbl

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred, pred_idx, probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {edit_lbl(pred)}; Probability: {probs[pred_idx]:.04f}'
btn_run.on_click(on_click_classify)
btn_run

In [74]:
widgets.VBox([widgets.Label('Select your dog!'), btn_upload, btn_run, out_pl, lbl_pred])

In [75]:
!pip install voila
!jupyter serverextension enable voila --sys-prefix